# Normal Distributions

In [ ]:
from matplotlib.patches import Ellipse
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import time

### One Dimension

In [ ]:
def normalpdf_1d(X,mean,sigma2):
    return np.exp(-0.5*np.square(X-mean)/sigma2)/np.sqrt(2*np.pi*sigma2)

In [ ]:
# create plot of 1d normal probability distribution function
npoint = 100
X = np.linspace(-5,5,npoint)
Z1 = normalpdf_1d(X,0,1)
Z2 = normalpdf_1d(X,0.5,4)
Z3 = normalpdf_1d(X,-1,0.25)

In [ ]:
# plot pdf
plt.figure()
plt.plot(X,Z1,"r-",label="$\mu=0$, $\sigma^2=1$")
plt.plot(X,Z2,"b-",label="$\mu=0.5$, $\sigma^2=4$")
plt.plot(X,Z3,"g-", label="$\mu=-1$, $\sigma^2=0.25$")
plt.legend()

### Multi-dimensional Normal PDF

In [ ]:
def normalpdf(X,mean,Sigma):
    d,nsample = X.shape
    Z = np.zeros((1,nsample))
    for i in range(nsample):
        Z[0,i] = np.exp(-0.5*np.matmul((X[:,[i]]-mean).T,np.matmul(np.linalg.inv(Sigma),X[:,[i]]-mean)))/np.sqrt(np.power(2*np.pi,d)*np.linalg.det(Sigma))
    return Z

### Surface Plot

In [ ]:
npoint = 100
Cov = np.array([[1,-0.5],[-0.5,2]])
mean = np.array([[0.5],[0.5]])
xmax = mean[0,0]+3
xmin = mean[0,0]-3
ymax = mean[1,0]+3
ymin = mean[1,0]-3
Xval,Yval = np.meshgrid(np.linspace(xmin,xmax,npoint),np.linspace(xmin,xmax,npoint))
X = np.concatenate((np.reshape(Xval,(1,npoint*npoint)),np.reshape(Yval,(1,npoint*npoint))),axis=0)
Z = normalpdf(X,mean,Cov)
Zval = np.reshape(Z,(npoint,npoint))

In [ ]:
fig = plt.figure()
ax = plt.axes(projection="3d")
ax.plot_surface(Xval,Yval,Zval,cmap=cm.jet)
ax.set_xlabel("X0")
ax.set_ylabel("X1")
ax.set_zlabel("Z")
ax.view_init(60, -30)

### Contours in x0-x1 plane

In [ ]:
# plot contours of normal pdf in 2d
fig,ax = plt.subplots()
ax.set_aspect("equal")
plt.contour(Xval,Yval,Zval)
U,Sigma,Vt = np.linalg.svd(Cov)
print("Sigma: {}".format(Sigma))

### Create contour using Matplotlib Ellipse

In [ ]:
fig,ax  = plt.subplots()
ax.set_aspect("equal")
Sigma0 = np.sqrt(Sigma[0])
Sigma1 = np.sqrt(Sigma[1])
ax.set_xlim(xmin,xmax)
ax.set_ylim(ymin,ymax)
# compute angle - convert from radians to degrees
angle = np.arctan(U[1][0]/(U[0,0]+1e-10))*180/np.pi
ellipse = Ellipse(xy=np.squeeze(mean),width=2*Sigma0,height=2*Sigma1,angle=angle, fill=False)
ax.add_patch(ellipse)

### Vectorized vs Non-Vectorized Normal PDF Function

In [ ]:
# non-vectorized - loop over each sample in data set X
def normalpdf(X,mean,Sigma):
    d,nsample = X.shape
    Z = np.zeros((1,nsample))
    for i in range(nsample):
        Z[0,i] = np.exp(-0.5*np.matmul((X[:,[i]]-mean).T,np.matmul(np.linalg.inv(Sigma),X[:,[i]]-mean)))/np.sqrt(np.power(2*np.pi,d)*np.linalg.det(Sigma))
    return Z

In [ ]:
# vectorized
def normalpdf_vectorized(X,mean,Sigma):
    d = X.shape[0]
    Z = np.exp(-0.5*(X-mean)*np.matmul(np.linalg.inv(Sigma),X-mean))/np.sqrt(np.power(2*np.pi,d)*np.linalg.det(Sigma))
    return Z

In [ ]:
nfeature = 5
nsample = 5000
X = np.random.randn(nfeature,nsample)
list_time_vectorized = []
list_time_nonvectorized = []
list_n = []
increment = 100
mean = np.random.randn(nfeature,1)
Covariance = np.diag(np.random.rand(5))
for count in range(1,int(nsample/increment)+1):
    n = count*increment
    list_n.append(n)
    time_start = time.time()
    Z1 = normalpdf(X[:,0:n],mean,Covariance)
    time_nonvectorized = time.time()
    list_time_nonvectorized.append(time_nonvectorized - time_start)
    Z2 = normalpdf_vectorized(X[:,0:n],mean,Covariance)
    list_time_vectorized.append(time.time() - time_nonvectorized)

fig = plt.figure()
plt.plot(list_n,list_time_nonvectorized,"r-",label="Non-Vectorized")
plt.plot(list_n,list_time_vectorized,"b-",label="Vectorized")
plt.legend()